In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load config & data 

In [6]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [7]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [8]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [9]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [10]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  139  0  0    495    472   465  0.05  0.37  0.49  0.80  0.74
1  147  1  0    487    467   457  0.05  0.43  0.51  0.73  0.65
2  152  2  0    474    457   448  0.05  0.45  0.53  0.67  0.58
3  161  3  0    453    436   426  0.05  0.47  0.54  0.61  0.53
4  139  4  0    428    414   406  0.05  0.47  0.58  0.58  0.45
5  150  5  0    404    393   382  0.06  0.47  0.59  0.67  0.52
6  155  6  0    388    381   371  0.06  0.42  0.58  0.57  0.42
7  154  7  0    374    371   361  0.06  0.46  0.60  0.51  0.35

 0 31943 316734.0 ! carry bought nice quality love perfect price - pockets
   0-1 10161 71706.0 cover color ! keyboard mac pro love air bottom hard
     0-1-1 4061 10374.0 cover bottom color keyboard apple purple cracked feet logo protector
     0-1-2 5103 6819.0 smell speck display kuzy retina smells chemical color chromebook incase
     0-1-3 983 906.0 reasonable stand perfect clean dont shipping gift experience find device
     0-1-4 14 11.0 samsonite base cracked cute folders house beautiful mobile impact recently
   0-2 7232 46694.0 handle straps months years strap shoulder broke work back year
     0-2-1 5635 8502.0 camera lenses pack gear compartments lens canon compartment heavy straps
     0-2-2 1597 2301.0 service refund cracked plastic started shipping terrible order pictures return
   0-3 14550 84120.0 & sleeve ; protection pocket inside ipad netbook air charger
     0-3-1 10266 14047.0 pocket phone compartment pockets pens lens room plenty cell camera
     0-3-2 4284 6187.